# Hospital Readmissions Data Analysis and Recommendations for Reduction - part of Capstone project for Coursera-IBM

### Background
In October 2016, the US government's Center for Medicare and Medicaid Services (CMS) began reducing Medicare payments for Inpatient Prospective Payment System hospitals with excess readmissions. Excess readmissions are measured by a ratio, by dividing a hospital’s number of “predicted” 30-day readmissions for heart attack, heart failure, and pneumonia by the number that would be “expected,” based on an average hospital with similar patients. A ratio greater than 1 indicates excess readmissions.

### Exercise Directions

In this exercise, we:
+ critique a preliminary analysis of readmissions data and recommendations (provided below) for reducing the readmissions rate. Based on public studies we proceeded with solid conclusions and based on the learnings of this course
+ construct a statistically sound analysis and make recommendations of my own (author of this study: Antonio Pesqueira

More instructions provided below. I will also include all the work and data **in this notebook and submit in my Github account**. 

### Resources
+ Data source: https://data.medicare.gov/Hospital-Compare/Hospital-Readmission-Reduction/9n3s-kdb3
+ More information: http://www.cms.gov/Medicare/medicare-fee-for-service-payment/acuteinpatientPPS/readmissions-reduction-program.html
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

All information of the public data and preliminary analysis can be found in the above links. As well the presentation of this study can be found in GitHub

Data Methodology

•	Analytical Approach: I would like to decide on the best approach to solve the problem and be able to analyse the correct data part of the exercise. The best model for the specific case might be a classification model to better assess the data and to understand which of them might be more relevant. 

•	Data Requirements: The data would need to in a tabular and structured format , where it will be required to have a unique identifier of the patient ID to better assess and cross-tabling the data between all sources.

•	Data Collection: Will be critical to better understand the patients admission, like: excessive inpatient days, admission history, recurring visits, and others.

•	Modeling and Evaluation: Once the model is under the modelling phase now we can collect the necessary feedback to understand the related evaluation. The following variables and data features now can be assessed.

•	Now the analysts can start sorting and defining the best model with better prediction rates and assess the rights models for hospital admissions. The analysts can start preparing the report to present to the stakeholders and sponsor in order to understand if the departments are ready to apply the model to real case scenarios and based on the training and testing results of the model. Now the report presents a concept for the evaluation of data modelling which is based on existing theoretical approaches and an empirical study conducted by the data science team. The main results of this report with respect to evaluation suggest to focus more on organizational issues of modelling and to better understand the potential areas of improvement, more on process instead of product quality, to consider different application scenarios of this data modelling exercise as well as to distinguish the enterprise-wide evaluation of data modelling as an organizational function from the evaluation of single projects using data modelling. This process step consists of recommendations for the evaluation procedure, persons involved, instruments, the design of important organizational dimensions as well as concrete process measures of the presented model.
****

In [ ]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import bokeh.plotting as bkp
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [ ]:
# to read in readmissions data 
hospital_read_df = pd.read_csv('data/cms_hospital_readmissions.csv')

****
## Preliminary Analysis

In [ ]:
# deal with missing and inconvenient portions of data 

clean_hospital_read_df = hospital_read_df[hospital_read_df['Number of Discharges'] != 'Not Available']
clean_hospital_read_df.loc[:, 'Number of Discharges'] = clean_hospital_read_df['Number of Discharges'].astype(int)
clean_hospital_read_df = clean_hospital_read_df.sort_values('Number of Discharges')

In [ ]:
# scatterplot for number of discharges vs. excess rate of readmissions
# lists work better with matplotlib scatterplot function
x = [a for a in clean_hospital_read_df['Number of Discharges'][81:-3]]
y = list(clean_hospital_read_df['Excess Readmission Ratio'][81:-3])

fig, ax = plt.subplots(figsize=(8,5))
ax.scatter(x, y,alpha=0.2)

ax.fill_between([0,350], 1.15, 2, facecolor='red', alpha = .15, interpolate=True)
ax.fill_between([800,2500], .5, .95, facecolor='green', alpha = .15, interpolate=True)

ax.set_xlim([0, max(x)])
ax.set_xlabel('Number of discharges', fontsize=12)
ax.set_ylabel('Excess rate of readmissions', fontsize=12)
ax.set_title('Scatterplot of number of discharges vs. excess rate of readmissions', fontsize=14)

ax.grid(True)
fig.tight_layout()

****

## Preliminary Report


**A. Initial observations based on the plot above**
+ Overall, rate of readmissions is trending down with increasing number of discharges
+ With lower number of discharges, there is a greater incidence of excess rate of readmissions (area shaded red)
+ With higher number of discharges, there is a greater incidence of lower rates of readmissions (area shaded green) 

**B. Statistics**
+ In hospitals/facilities with number of discharges < 100, mean excess readmission rate is 1.023 and 63% have excess readmission rate greater than 1 
+ In hospitals/facilities with number of discharges > 1000, mean excess readmission rate is 0.978 and 44% have excess readmission rate greater than 1 

**C. Conclusions**
+ There is a significant correlation between hospital capacity (number of discharges) and readmission rates. 
+ Smaller hospitals/facilities may be lacking necessary resources to ensure quality care and prevent complications that lead to readmissions.

**D. Regulatory policy recommendations**
+ Hospitals/facilties with small capacity (< 300) should be required to demonstrate upgraded resource allocation for quality care to continue operation.
+ Directives and incentives should be provided for consolidation of hospitals and facilities to have a smaller number of them with higher capacity and number of discharges.

In [ ]:
# start with inferential analysis
clean_hospital_read_df

In [ ]:
x

In [ ]:
len(x)

In [ ]:
y

In [ ]:
len(y)

One of the conclusions of the preliminary analysis is that there is a significant correlation between number of discharges and excess readmission rate.  Let us test that using linear regression and a one-sample t-test.

H0:  There is no correlation between discharges and excess readmission rate (i.e. slope is 0).

H1:  There is a correlation between discharges and excess readmission rate (slope not equal to 0).

Let's run our linear regression.

In [ ]:
from scipy.stats import linregress
linregress(x, y)

Our p-value of approximately 1.50 * 10^-23 is extremely small--much smaller than our 1% significance level--so we reject the null hypothesis.  There is a correlation between number of discharges and excess readmission ratio.  Is this correaltion negative?  Using 11492 degrees of freedom, our critical t-score for .01 significance is 2.576.  Let's now construct a 99% confidence interval.  

In [ ]:
slope, intercept, r_value, p_value, std_err = linregress(x, y)
t = 2.576

In [ ]:
conf_high = slope + (t*std_err)
conf_low = slope - (t*std_err)
print "99% Confidence Interval: (",conf_low,",",conf_high,")" 

0 is not in our 99% confidence interval, so our slope value is statistically signficant.  We are extremely confident that the correlation between number of discharges and excess readmission rate is negative.

Now let's test one of the other conclusions that hospitals with small capacities have higher excess readmission rates and, therefore, need upgraded resource allocation.  We will use a one-sided, two-sample Z-test with 1% signficance, testing the mean excess readmission rates mentioned above (discharges < 100 and discharges >1000).

H0:  There is no difference in excess readmission rates between hospitals that discharge < 100 and those that discharge > 1000.

H1:  There is a difference, and excess readmission rates are higher for hospitals that discharge < 100.

The means are noted above, but let's recalculate them and their standard deviations.

In [ ]:
df = clean_hospital_read_df[81:-3]
df

In [ ]:
low_df = df['Excess Readmission Ratio'][df['Number of Discharges'] < 100]
low_df

In [ ]:
low_df.mean()

In [ ]:
low_df.std()

In [ ]:
len(low_df)

In [ ]:
high_df = df['Excess Readmission Ratio'][df['Number of Discharges'] > 1000]
high_df

In [ ]:
high_df.mean()

In [ ]:
high_df.std()

In [ ]:
len(high_df)

Now let's run our Z-test.

In [ ]:
import math
diff = low_df.mean()-high_df.mean()-0
s = math.sqrt((((low_df.std())**2)/len(low_df)) + (((high_df.std())**2)/len(high_df)))
z = diff/s
z

For .01 significance, the critical z-value is 2.325. Our calculated z-value far exceeds that, so we reject the null hypothesis.  The excess readmission ratio for hospitals that discharge < 100 is higher than the excess readmission ratio for hospitals that discharge > 1000.  Therefore, I support the conclusions and policy recommendations.  Low capacity hospitals should be given upgraded resources to lower excess readmission ratios.  In this case, statistical signficance aligns with practical signifcance.

The scatter plot from the presentation in pdf does an effective job at visualized the distribution of excess readmission rates among different discharge rates.  However, it is hard to see correlation.  The plots could benefit by showing histograms for hospitals of different capacities.

In [ ]:
fig, ax = plt.subplots()
ax.hist(low_df, bins='auto')
ax.set_xlabel('Excess Readmission Rate', fontsize = 12)
ax.set_title('Excess rate of readmissions for discharges < 100', fontsize=14)

In [ ]:
fig, ax = plt.subplots()
ax.hist(high_df, bins='auto')
ax.set_xlabel('Excess Readmission Rate', fontsize = 12)
ax.set_title('Excess rate of readmissions for discharges > 1000', fontsize=14)

One can get a better sense of the distributions among higher lower discharge rates.

Thank you. Antonio Pesqueira

Please dont forget to see all graphs and presentation in pdf from my GitHub. As the data is not fully public I cannot share the csv file.